# Chapter 11: Gettin' Nerdy with It: Advanced Power Analysis

This is the companion notebook to Chapter 11 of the Hardware Hacking Handbook by Jasper van Woudenberg and Colin O'Flynn. The headings in this notebook follow the headings in the book.

© 2021. This work is licensed under a [CC BY-SA 4.0 license](https://creativecommons.org/licenses/by-sa/4.0/). 

## Resynchronization

In [ ]:
import numpy as np
import pylab as py

# Helper for plotting
def trcplot(trace, title="", xlabel="", ylabel="", label="", axs=None):
    if not axs:
        fig, axs = py.subplots()
    axs.set_title(title)
    axs.set_xlabel(xlabel)
    axs.set_ylabel(ylabel)
    axs.plot(trace, label=label)

In [ ]:
# Calculate the Sum of Abs Differences between 
# - the reference vector ref_data, 
# - shifted over all indices of inp_data 
# - indicated in window
def findSAD(inp_data, ref_data, window):        
    # Initialize emtpy array
    sadarray = np.empty(window[1]-window[0])
    
    # For every index in the window
    for ptstart in range(window[0], window[1]):
        # Calculate the Sum of Abs differences between ref_data and inp_data
        sadarray[ptstart-window[0]] = np.sum(
            np.abs(inp_data[ptstart:(ptstart+len(ref_data))]-ref_data)
         )
        
    # Result!
    return sadarray

In [ ]:
REF_TRACE = 0               # Trace index of reference trace
REF_PATTERN = range(25,35)  # Location of the reference pattern
WINDOW = (15, 35)           # Set the window to sweep over
    
# Load four traces that aren't synchronized
testdata = np.load('traces/unsync_traces.npy')
# The output traces will be written here
newtraces = np.zeros(np.shape(testdata))
# Select reference points from one trace
refpts = testdata[REF_TRACE][REF_PATTERN]

# Find location of minimum in reference trace
sad = findSAD(testdata[REF_TRACE], refpts, WINDOW)
refminloc = np.argmin(sad)

# Sweep over each trace
for i in range(0, 4):
    # Calculate SAD over window
    sad = findSAD(testdata[i], refpts, WINDOW)    
    # Find offset which minimizes the SAD
    minloc = np.argmin(sad)    
    # Shift trace as needed
    shift = minloc-refminloc
    # Pad where needed
    if shift < 0:
        newtraces[i] = np.append(np.zeros(-shift), testdata[i][:shift])
    else:
        newtraces[i] = np.append( testdata[i][shift:], np.zeros(shift))

In [ ]:
# Show the results!
fig, axs = py.subplots(2)
for i in range(1,len(testdata)):
    trcplot(testdata[i], "Misaligned", axs=axs[0])
    trcplot(newtraces[i], "Aligned", axs=axs[1])
py.savefig("fig/11_align.svg", format="svg")